# MLOps : Quand le Machine Learning rencontre l’esprit du DevOps

## Introduction

Les projets de Machine Learning (ML) sont souvent comparés à des expériences scientifiques : on explore, on teste, on ajuste. Une sorte de laboratoire où chaque idée est mise à l’épreuve. Mais cette approche, bien qu’efficace pour apprendre, montre vite ses limites quand il s’agit de passer à la production.

Avec des modèles plus complexes, plus de données, et des besoins de monitoring, il ne suffit plus que « ça marche dans le notebook ». C’est ici que le MLOps, inspiré du DevOps, entre en jeu : il structure, automatise et industrialise chaque étape d’un projet ML. Pour le démontrer, prenons un exemple concret avec un **projet d’analyse de sentiments**. Il s'agit de créer un prototype d'IA pour prédire le sentiment des tweets à partir de données open source et déployé le model via une API Cloud.

## 1) Premier pas

L'une des premières étapes est le choix du modèle, c'est une étape essentielle mais souvent complexe. Cela dépend des spécificités des données, des contraintes du projet et des objectifs visés. Une approche pragmatique consiste à commencer par un modèle simple, puis à explorer des modèles de plus en plus avancés.

Pour notre projet d'analyse de sentiment, nous commençons par **une régression logistique**. Pourquoi ? Parce qu’elle est rapide, facile à implémenter et souvent suffisante pour valider une idée. Avec cette méthode, nous avons utilisé des caractéristiques textuelles de base, comme la fréquence des mots (bag of words). Cette **baseline** joue un double rôle : elle permet de valider rapidement la faisabilité de l'approche et sert de référence pour évaluer les améliorations apportées par des modèles plus complexes.

Une fois que notre régression logistique est en place et que nous avons une baseline, il devient crucial de structurer notre approche pour garantir la reproductibilité des résultats, faciliter la comparaison des expérimentations, et préparer le modèle au déploiement en production, ce que permettent les principes du MLOps et des outils comme **MLflow**. **MLflow** stocke les métadonnées de run et les modèles, tout en assurant le suivi des expérimentations.



En ce qui concerne MLops, nous allons utiliser conjointement plusieurs outils :
- **MLflow**: stocke les métadonnées de run et les modèles.
- **DVC**: stocke et versionne les datasets, sans multiplier l’espace à chaque run. DVC est conçu pour gérer efficacement les changements et éviter de télécharger ou de pousser inutilement des données si elles n'ont pas été modifiées.
- **Git**: stocke les scripts, configurations et fichiers .dvc nécessaires pour synchroniser code et données, assurant une traçabilité complète du projet.
- **Docker/Conda**: Garantie d'avoir un environnement reproductible. 
    - ex : conda pack -n mlops-env -o mlops-env.tar.gz && docker build -f Dockerfile.mlops-env -t mlops-env:v1 .

Ensemble, ces outils créent un écosystème intégré qui garantit la reproductibilité des résultats en synchronisant chaque version du code, des données et des modèles. Ils facilitent également la collaboration entre équipes en offrant un suivi clair des expérimentations et des changements, tout en structurant des pipelines de Machine Learning robustes et facilement déployables à l'échelle.



### Points clés :
- Utilisation de Docker
- L'utilisation de DVC et git permet de faire en sorte que le script 
- l'utilisation d'Hydra permet de changer de passer d'une expérience à autre simplement.

### Résultats :
Voici les performances obtenues sur notre dataset :
- Précision : 76,5%
- F1-score : 74,2%

## Création du pipeline avec DVC (Data Version Control)

Dans un pipeline DevOps classique, vous codez, vous poussez sur Git, et boum, tout se construit et se déploie automatiquement. En MLOps, on va un cran plus loin, les étapes classiques sont : 
- Préparer les données (nettoyage, enrichissement).
- Entraîner les modèles (et tester plusieurs configurations).
- Valider les résultats (cross-validation, ensembles de test).
- Et ensuite ? Surveiller en continu. Parce qu’un modèle, contrairement à du code, peut se dégrader avec le temps !(drift)

En machine learning, les pipelines impliquent non seulement du code, mais aussi des données, des modèles, et des étapes de transformation interconnectées. C'est là que DVC devient un outil indispensable. Inspiré de Git, DVC détecte automatiquement les changements dans votre pipeline, ne réexécutant que les étapes nécessaires pour garantir une gestion efficace et reproductible des workflows ML.


## Monter en puissance avec des modèles avancés

Bien que ce soit un bon point de départ, le modèle manque de finesse pour capturer des relations contextuelles. Pour améliorer les résultats, nous avons besoin de tester des modèles plus avancés, comme des word embeddings combinés à des réseaux neuronaux (CNN ou RNN). Ces approches permettent de mieux comprendre le contexte des mots en tenant compte des séquences.

Avec Hydra, il devient facile d'ajouter des configurations sans trop toucher aux scripts python.

Capture d’écran des résultats et du graphe des erreurs communes.

Résultat avec un CNN :
- Précision : 82,7%
- F1-score : 81,4%

Cette amélioration montre que le modèle capture désormais des nuances plus subtiles dans les sentiments. Cependant, la complexité augmente : le modèle prend plus de temps à entraîner, et le suivi des hyperparamètres devient critique.

Capture d’écran des résultats comparés aux erreurs de la régression logistique.

## Étape 3 : L’État de l’art avec BERT

Enfin, nous avons utilisé BERT, un modèle préentraîné d’État de l’art, capable de comprendre les subtilités linguistiques et les relations complexes dans les phrases. Avec BERT, les résultats ont été impressionnants :

Résultat avec BERT :
- Précision : 89,5%
- F1-score : 88,9%

Cependant, l’utilisation de BERT a introduit de nouveaux défis :
- Temps d’entraînement prolongé : Il a fallu 10 fois plus de temps que pour la régression logistique.
- Infrastructure nécessaire : L’entraînement a nécessité un GPU, et le suivi des versions de données et de modèles est devenu encore plus crucial.

Capture d’écran des résultats, avec une visualisation des prédictions BERT sur des exemples complexes.

## Passage en production avec MLOps

À chaque étape, les modèles deviennent plus performants, mais aussi plus complexes à gérer. C’est ici que le MLOps prend tout son sens. Voici comment il a structuré notre projet :

### 1. Suivi des expérimentations

Grâce à des outils comme MLflow, nous avons enregistré chaque expérimentation :
- Version des données utilisées.
- Hyperparamètres et configurations des modèles.
- Performances obtenues (précision, F1-score, etc.).

Cela nous a permis de comparer les résultats facilement et de justifier nos choix à chaque étape.

### 2. Automatisation des pipelines

Nous avons automatisé les étapes clés :
- Préparation des données (nettoyage, transformation).
- Entraînement des modèles avec différentes configurations.
- Validation sur des ensembles indépendants.

Un outil comme Kubeflow nous a aidés à orchestrer ces tâches et à les rendre reproductibles.

### 3. Monitoring en production

Une fois BERT déployé, un système de monitoring a été mis en place pour surveiller les performances en temps réel. Par exemple :
- Détection des drifts de données : Si les avis clients évoluent (argot, emojis, etc.).
- Suivi des métriques : Si le modèle perd en précision ou en rappel.

Conclusion : Apprendre, structurer, et industrialiser

L’histoire de ce projet d’analyse de sentiments montre bien la double nature des projets ML. D’un côté, il y a l’aspect exploratoire : tester différents modèles et approches pour trouver ce qui fonctionne. De l’autre, il y a le besoin d’industrialisation, où la reproductibilité et la fiabilité deviennent essentielles.

Le MLOps, en s’appuyant sur les principes du DevOps, nous a permis de structurer cette transition. Il a transformé une série d’expérimentations en une solution robuste, prête pour la production. Et à chaque étape, il nous a offert des outils pour gérer la complexité croissante du Machine Learning.

# MLOps : Construire des modèles qui tiennent la distance

Développer un projet de machine learning suit généralement un parcours bien défini : collecte et préparation des données, expérimentation et entraînement des modèles, évaluation des performances, puis mise en production. Chaque étape apporte son lot de défis techniques et méthodologiques, mais avec une bonne organisation, il est possible d’obtenir un modèle performant et exploitable.

Cependant, à mesure que le projet évolue et prend de l’ampleur, de nouvelles contraintes apparaissent. Comment garantir que le modèle reste performant dans le temps ? Comment assurer la reproductibilité des résultats ? Comment maintenir un cadre structuré malgré l’évolution des données, des infrastructures ou des équipes ?

C’est ici que MLOps intervient. Son rôle est de structurer et fiabiliser le cycle de vie des modèles en anticipant ces défis avant qu’ils ne deviennent des freins majeurs. Il permet d’assurer la traçabilité, la reproductibilité et l’évolutivité des modèles, indépendamment des changements qui surviennent au fil du temps. Plutôt que de simplement optimiser la vitesse de mise en production, MLOps vise à garantir une progression stable et maîtrisée, en réduisant les risques liés aux infrastructures, aux données et à la gouvernance.

Là où un projet ML peut devenir difficile à maintenir sur le long terme, MLflow et DVC apportent des solutions concrètes :

- MLflow centralise le suivi des expérimentations et la gestion des modèles, facilitant leur versioning et leur évaluation.
- DVC assure une gestion efficace des données et des pipelines, garantissant la reproductibilité des entraînements et des résultats.

Ces outils permettent non seulement de structurer le travail, mais aussi d’éviter la dépendance aux individus, en garantissant que chaque avancée est documentée et exploitable par toute l’équipe, même après plusieurs mois d’inactivité sur le projet.

💡 Pour illustrer ces bénéfices concrets, nous prendrons l’exemple d’une analyse de sentiments basée sur des tweets. Ce projet nous servira de fil conducteur pour montrer comment MLflow, DVC et Docker/Kubernetes répondent aux défis techniques et organisationnels tout au long du cycle de vie d’un modèle. De la gestion des données à la mise en production, chaque étape sera explorée sous l’angle du MLOps, afin de démontrer comment cette approche s’impose aujourd’hui comme un levier stratégique pour garantir la pérennité et la scalabilité des solutions d’intelligence artificielle.

## Analyse exploratoire des données (AED)


| **Composant MLOps** | **Solution sans MLOps** | **Problème potentiel à long terme** | **Solution MLOps** | **Outil** |
|----------------------|----------------------|----------------------|----------------------|----------------------|
| **Analyse exploratoire des données (AED)** | Analyse et visualisation ponctuelle des données via des notebooks | Pas de traçabilité des transformations appliquées, difficulté à reproduire l’analyse | Versionner les transformations et standardiser le préprocessing | **DVC** (versionning des données), **MLflow** (tracking des expériences) |
| **Préparation des données et extraction de caractéristiques** | Nettoyage et transformation des données manuellement, stockage en local ou sur un drive | Incohérence des datasets entre équipes, difficulté à reproduire les jeux de données | Versionnement des données et pipelines reproductibles | **DVC** (gestion des datasets), **MLflow** (enregistrement des transformations) |
| **Entraînement et réglage du modèle** | Expérimentations non structurées dans des notebooks, hyperparamètres ajustés manuellement | Perte des versions intermédiaires des modèles, pas de reproductibilité des expériences | Tracking des entraînements, gestion des hyperparamètres, reproductibilité | **MLflow** (Tracking, Model Registry), **DVC** (pipelines de traitement) |
| **Revue et gouvernance du modèle** | Validation et revue manuelle des modèles, critères de qualité non formalisés | Risque de biais non détectés, modèles non conformes aux normes réglementaires | Automatisation des tests de validation, auditabilité des modèles | **MLflow** (validation et audit des modèles), **DVC** (traçabilité des données) |
| **Inférence et déploiement du modèle** | Déploiement manuel via scripts, modèle stocké sans versioning | Risque d’erreurs de configuration, difficulté à restaurer une version stable | Automatisation du packaging et du déploiement, versionnement des modèles | **MLflow** (Model Registry, Serving), **Docker/Kubernetes** (déploiement scalable) |
| **Surveillance des modèles** | Pas de monitoring, ou vérification manuelle des prédictions | Dégradation des performances non détectée, modèle obsolète face à de nouvelles données | Monitoring automatique des performances et détection du drift | **MLflow** (Model Monitoring), **Prometheus/Grafana** (visualisation des métriques) |
| **Réentraînement automatisé du modèle** | Réentraînement manuel, sans déclenchement basé sur des critères précis | Modèle vieillissant, performance en baisse, absence de mise à jour proactive | Déclenchement du réentraînement automatique basé sur des métriques prédéfinies | **MLflow** (gestion des modèles), **DVC** (pipelines automatisés) |


## Cycle de vie

## MLflow : Le carnet de bord des expérimentations ML

(# Ici, vous pouvez insérer une capture d'écran MLflow ou un court code montrant l'utilisation de mlflow.start_run(), mlflow.log_param(), mlflow.log_metric()...)

L'une des premières étapes dans un projet ML est le choix du modèle. Cette étape essentielle est souvent complexe et dépend autant des spécificités des données que des contraintes du projet et des objectifs visés. Pour démarrer, il est judicieux de commencer par un modèle simple, puis d’explorer des modèles plus avancés. Dans notre cas, nous avons opté pour une régression logistique. Pourquoi ? Parce qu’elle est rapide, facile à implémenter et dsouvent suffisante pour valider la faisabilité de l’approche. Ce modèle, que l’on nomme baseline, joue un double rôle :
- Confirmer rapidement la validité de la démarche.
- Servir de référence pour évaluer les gains apportés par des modèles plus sophistiqués.

### Capturer les métriques

Dans le cadre de ce projet d’analyse de sentiments, la baseline (régression logistique) est enregistrée dans le Model Registry de MLflow, ce qui offre un suivi centralisé de ses performances.
Comparaison de la baseline avec un modèle plus avancé

Lorsque de nouveaux modèles sont testés (par exemple un réseau de neurones), MLflow facilite la comparaison avec la baseline, tant au niveau des métriques que des hyperparamètres utilisés. Un simple coup d’œil à l’interface permet de voir si le nouveau modèle surpasse ou non la baseline initiale.

##  Un pipeline unique ou plusieurs ?

(# Rajouter éventuellement un bref aperçu YAML/Hydra montrant comment vous paramétrez le vectoriseur et le modèle.)

Cependant, la baseline n’est qu’un point de départ : à mesure que l’on itère, que l’on ajuste les hyperparamètres et que l’on évalue différents modèles, il devient primordial de garder une traçabilité précise de chaque expérimentation. Dans notre projet, nous souhaitions par exemple tester plusieurs vectoriseurs (TF-IDF, GloVe, BERT…), différents modèles (régression logistique, LSTM, Random Forest, etc.) ou encore des configurations variées (batch_size, lr, seuils de découpage des données, etc.).

Pour éviter de multiplier les scripts et de rendre la maintenance laborieuse, on s’appuie sur un pipeline modulaire et paramétrable. Naïvement, deux approches sont possibles :
- Dupliquer le pipeline pour chaque variante,
- Créer un pipeline modulaire et paramétrable, où la vectorisation, le modèle ou les hyperparamètres sont gérés via un système de configuration (ex. Hydra, YAML, etc.).

La première approche peut paraître simple mais devient vite coûteuse en maintenance et en risques de divergence de code (copies multiples). Le guide Google “Continuous delivery and automation pipelines in machine learning” (souvent cité comme une référence MLOps) préconise ainsi :

    “To reduce complexity and promote reuse, we recommend building modular and parametric pipelines. Each stage (data processing, training, evaluation) is defined once and can be reused by multiple variants or parameter sets. This avoids duplication of code and simplifies maintenance.”

En d’autres termes, un pipeline modulaire et paramétrable reste plus souple, plus facile à tester et à maintenir, tout en assurant la traçabilité au sein d’une même structure de code. Grâce à une configuration centralisée, vous n’avez qu’à adapter quelques paramètres pour changer de vectoriseur ou de modèle, ce qui accélère considérablement l’expérimentation et limite les erreurs.

### CI/CD Github
(# Rajoutez une courte phrase expliquant ce qu’on voit dans la capture d’écran, par ex. “Ci-dessous un exemple de workflow GitHub Actions déclenché à chaque push, qui exécute nos tests et notre pipeline MLOps.”)
Capture d'écran

## Passage en production

(# Ici, vous pouvez mentionner comment vous packagez le modèle, par exemple via Docker. Ou comment vous utilisez un orchestrateur type Kubernetes, ou un service. Expliquer très brièvement la démarche.)

Après avoir validé un modèle satisfaisant, il faut le déployer pour qu’il soit accessible en production. Cette étape peut impliquer :
- La containerisation (via Docker) pour embarquer le modèle et son environnement d’exécution,
- Le déploiement sur un cluster Kubernetes ou un service d’hébergement (AWS, GCP, Azure…),
- La mise en place d’une API (REST, gRPC) permettant de recevoir des données et de retourner des prédictions,
- Des scripts ou workflows CI/CD (GitHub Actions, GitLab CI, Jenkins…) assurant que chaque nouvelle version du modèle est correctement testée et livrée.

(# Ajouter un exemple rapide : “Nous avons créé un Dockerfile décrivant l’environnement, qui inclut Python 3.X, scikit-learn, MLflow, etc. Ensuite, un script de déploiement…”)
DVC et MLflow : Un duo pour une gestion cohérente des modèles et des données

Après avoir défini une baseline et exploré des modèles plus avancés, la gestion des données et des versions devient essentielle. Il ne suffit pas de versionner les modèles avec MLflow ou les données avec DVC, mais de garantir un lien explicite entre les deux. Cette association assure que chaque modification repose sur un environnement traçable et reproductible, une pierre angulaire du MLOps.

Prenons un exemple concret : reprendre un modèle tagué en préproduction dans MLflow, restaurer son environnement exact, et créer une nouvelle branche Git pour préparer une mise à jour.
```bash
# Récupérer le hash Git depuis MLflow
mlflow models describe -m "models:/sentiment-analysis-baseline/Production" > model_metadata.txt
GIT_COMMIT=$(grep "git_commit" model_metadata.txt | awk -F': ' '{print $2}')

# Restaurer l'environnement
git checkout $GIT_COMMIT
dvc checkout
dvc pull
```

(# Expliquer en quelques mots ce qui se passe dans le script. “mlflow models describe...” récupère le commit Git associé au modèle, on fait ensuite git checkout pour retrouver le code, dvc checkout et dvc pull pour synchroniser les données.”)

## Conclusion

(# Vous pouvez illustrer comment le monitoring ou la dérive de données peut être détecté, ou simplement raccourcir à votre convenance.)

En définitive, l’approche MLOps va bien au-delà du simple « ça marche dans le notebook ». En outillant chaque étape (tracking des expériences, versioning des données, CI/CD et déploiement), elle garantit traçabilité, collaboration et industrialisation. Dans l’exemple de l’analyse de sentiments, l’approche modulaire et paramétrable permet d’itérer rapidement sur différents modèles ou configurations sans perdre le fil. Résultat : on allie la vitesse d’exploration scientifique à la fiabilité d’un environnement de production, assurant la pérennité et l’impact des projets ML.

(# Pensez à ajouter éventuellement un glossaire de termes techniques ou un court paragraphe “Aller plus loin” listant des liens officiels : MLflow, DVC, Hydra, Docker, etc. Cela peut aussi améliorer la lisibilité et l’enrichissement pour le lecteur.)
